###  The following tasks come from one of the Udemy courses where they were completed using Python pandas. I've also done them using Postgresql.

In [1]:
# import libraries
import pandas as pd

In [16]:
# importing the data frame
df=pd.read_csv('/home/kinga/python/Postgresql_analysis_baby_names/us_baby_names.csv')

In [18]:
df.sort_values(by='Year', ascending=False)

,Year,Name,Gender,Count
1957045,2018,Zzyzx,M,5
1935695,2018,Cory,F,9
1935682,2018,Ceanna,F,9
1935683,2018,Celes,F,9
1935684,2018,Cereniti,F,9
...,...,...,...,...
1339,1880,Randolph,M,19
1340,1880,Stewart,M,19
1341,1880,Vern,M,19
1342,1880,Wilmer,M,19


In [19]:
df.columns=['yr', 'name', 'gender', 'number']

In [21]:
df.head()

,yr,name,gender,number
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


### Importing data into Postgresql using Pythona

In [2]:
# inserting data into table 'names_popularity'
from sqlalchemy import create_engine
engine = create_engine('postgresql://kinga@localhost:5432/baby_names')

In [ ]:
df.to_sql('names_popu1larity', engine, if_exists='replace',index=False) 

In [25]:
# checking if the data has been loaded correctly
query="""SELECT * FROM names_popularity FETCH FIRST 5 ROWS ONLY"""
query=pd.read_sql_query(query, engine)
query

,yr,name,gender,number
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [26]:
query1="""SELECT COUNT(*) FROM names_popularity"""
query1=pd.read_sql(query1, engine)
query1

,count
0,1957046


In [ ]:
# removing rows with year<1970
delete from names_popularity
where yr<1970

## Data analysis

### 1. 10 most popular female and male names in 2018

In [3]:
popular2018="""SELECT * FROM 
                (SELECT yr, name, number,gender, rank() over(ORDER BY number DESC) FROM names_popularity
                WHERE yr=2018 AND gender='F'
                ORDER BY number DESC
                FETCH FIRST 10 ROWS ONLY) as tab1
                
                UNION ALL
                
                SELECT * FROM 
                (SELECT yr, name, number, gender, rank() over (ORDER BY number DESC) FROM names_popularity
                WHERE yr=2018 AND gender='M'
                ORDER BY number DESC
                FETCH FIRST 10 ROWS ONLY) as tab2
                """
popular2018=pd.read_sql(popular2018, engine)
popular2018

,yr,name,number,gender,rank
0,2018,Emma,18688,F,1
1,2018,Olivia,17921,F,2
2,2018,Ava,14924,F,3
3,2018,Isabella,14464,F,4
4,2018,Sophia,13928,F,5
5,2018,Charlotte,12940,F,6
6,2018,Mia,12642,F,7
7,2018,Amelia,12301,F,8
8,2018,Harper,10582,F,9
9,2018,Evelyn,10376,F,10


### 2. Male names that were in the group of 15 most popular names in 1980 and 2018

In [13]:
Mpopular_1980_2018="""SELECT * FROM 
                            (SELECT yr, name, number FROM names_popularity
                            WHERE yr=1980 AND gender='M'
                            ORDER BY NUMBER DESC
                            FETCH FIRST 15 ROWS ONLY) names_1980
                    INNER JOIN 
                            (SELECT yr, name, number FROM names_popularity
                            WHERE yr=2018 AND gender='M'
                            ORDER BY NUMBER DESC
                            FETCH FIRST 15 ROWS ONLY) names_2018
                    ON names_1980.name=names_2018.name """
Mpopular_1980_2018=pd.read_sql(Mpopular_1980_2018, engine)
Mpopular_1980_2018

,yr,name,number,yr,name,number
0,1980,Michael,68696,2018,Michael,11620
1,1980,James,39325,2018,James,13525
2,1980,Daniel,29894,2018,Daniel,11173
3,1980,William,25654,2018,William,14516


In [ ]:
# the same results as above can be achieved using teporary tables:
CREATE temp table tab1
as SELECT yr, name, number FROM names_popularity
WHERE yr=1980 AND gender='M'
ORDER BY NUMBER DESC
FETCH FIRST 20 ROWS ONLY

CREATE temp table tab2
as SELECT yr, name, number FROM names_popularity
WHERE yr=2018 AND gender='M'
ORDER BY NUMBER DESC
FETCH FIRST 20 ROWS ONLY

SELECT tab1.yr, tab1.name, tab1.number, tab3.yr, tab2.name, tab2.number
FROM tab1 INNER JOIN tab2
ON tab1.name=tab2.name

### 3. Female names that were in the group of 15 most popular names in 1980 and 2018

In [14]:
Fpopular_1980_2018="""SELECT names_1980.name,  names_1980.number as count_1980, names_2018.number as count_2018  
                    FROM 
                        (SELECT yr, name, number FROM names_popularity
                        WHERE yr=1980 AND gender='F'
                        ORDER BY NUMBER DESC
                        FETCH FIRST 15 ROWS ONLY) names_1980
                    INNER JOIN 
                        (SELECT yr, name, number FROM names_popularity
                        WHERE yr=2018 AND gender='F'
                        ORDER BY NUMBER DESC
                        FETCH FIRST 15 ROWS ONLY) names_2018
                    ON names_1980.name=names_2018.name """
Fpopular_1980_2018=pd.read_sql(Fpopular_1980_2018, engine)
Fpopular_1980_2018

,name,count_1980,count_2018
0,Elizabeth,19528,8513


### 4. Setting the following parameters:
- **total_count**- total a number of times the name has appeared
- **first_year** - the year the name first appeared
- **last_year** - the year the name last appeared
- **number_of_years** - the number of years in which the name appeared 
- **max_count** - the maximum number of name's appearance
- **best_year** - the year in which the name was given the most often

In [16]:
data_aggregation="""SELECT name, gender, sum(number) as total_count, min(yr) as first_year, 
                    max(yr) as last_year, count(number) as number_of_years, max(number) as max_count 
                    FROM names_popularity GROUP BY name, gender """
data_aggregation=pd.read_sql(data_aggregation, engine)
data_aggregation.head(10)


,name,gender,total_count,first_year,last_year,number_of_years,max_count
0,Aaban,M,114.0,2007,2018,11,16
1,Aabha,F,35.0,2011,2016,5,9
2,Aabid,M,16.0,2003,2018,3,6
3,Aabidah,F,5.0,2018,2018,1,5
4,Aabir,M,10.0,2016,2018,2,5
5,Aabriella,F,38.0,2008,2018,6,11
6,Aada,F,13.0,2015,2018,2,8
7,Aadam,M,273.0,1987,2018,27,22
8,Aadan,M,130.0,2003,2017,11,23
9,Aadarsh,M,209.0,2000,2018,18,18


- aggregation similar to above one but I also search for the year in which the name was given the most often

In [9]:
data_aggregation2="""select aggregated.name, aggregated.gender, total_count, first_year, 
                    last_year, number_of_years, max_count, best_year
                    FROM 
                        (SELECT name, gender, sum(number) as total_count, min(yr) as first_year, 
                        max(yr) as last_year, count(number) as number_of_years, max(number) as max_count 
                        FROM names_popularity GROUP BY name, gender) aggregated

                    INNER JOIN
                        (SELECT name, gender, best_year from (select name, gender, yr as best_year, 
                        rank() OVER (PARTITION BY gender, name ORDER BY number DESC ) as ranked 
                        FROM names_popularity) ranking where ranked=1) b_year
                    ON aggregated.name=b_year.name
                    AND aggregated.gender=b_year.gender"""
data_aggregation2=pd.read_sql(data_aggregation2, engine)
data_aggregation2.head(10)

,name,gender,total_count,first_year,last_year,number_of_years,max_count,best_year
0,Aaban,M,114.0,2007,2018,11,16,2014
1,Aabha,F,35.0,2011,2016,5,9,2014
2,Aabid,M,16.0,2003,2018,3,6,2018
3,Aabidah,F,5.0,2018,2018,1,5,2018
4,Aabir,M,10.0,2016,2018,2,5,2016
5,Aabir,M,10.0,2016,2018,2,5,2018
6,Aabriella,F,38.0,2008,2018,6,11,2016
7,Aada,F,13.0,2015,2018,2,8,2018
8,Aadam,M,273.0,1987,2018,27,22,2015
9,Aadan,M,130.0,2003,2017,11,23,2009


- the same as above but with filtering choosen names-in this case 'Mary', 'Elisabeth', 'Emma'

In [10]:
data_aggregation3="""select aggregated.name, aggregated.gender, total_count, first_year, last_year, 
                    number_of_years, max_count, best_year
                    FROM 
                        (SELECT name, gender, sum(number) as total_count, min(yr) as first_year, 
                        max(yr) as last_year, count(number) as number_of_years, max(number) as max_count 
                        FROM names_popularity GROUP BY name, gender) aggregated

                    INNER JOIN
                        (SELECT name, gender, best_year from (select name, gender, yr as best_year, 
                        rank() OVER (PARTITION BY gender, name ORDER BY number DESC ) as ranked 
                        FROM names_popularity) ranking where ranked=1) b_year
                    ON aggregated.name=b_year.name
                    AND aggregated.gender=b_year.gender
                    WHERE aggregated.name IN ('Mary','Elisabeth', 'Emma') 
                    ORDER BY aggregated.name  """
data_aggregation3=pd.read_sql(data_aggregation3, engine)
data_aggregation3

,name,gender,total_count,first_year,last_year,number_of_years,max_count,best_year
0,Elisabeth,F,30759.0,1970,2018,49,1093,2001
1,Elisabeth,M,27.0,1979,1989,4,10,1989
2,Emma,F,431900.0,1970,2018,49,22706,2003
3,Emma,M,596.0,1971,2018,37,102,2004
4,Mary,F,367194.0,1970,2018,49,19203,1970
5,Mary,M,1608.0,1970,2014,41,99,1970


### 5. 10 male and female names that where the most popular during the whole analysed period of time

In [11]:
best_all_times="""WITH cte as
                    (select aggregated.name, aggregated.gender, total_count, first_year, 
                    last_year, number_of_years, max_count, best_year
                    FROM 
                        (SELECT name, gender, sum(number) as total_count, min(yr) as first_year, 
                        max(yr) as last_year, count(number) as number_of_years, max(number) as max_count 
                        FROM names_popularity GROUP BY name, gender) aggregated

                    INNER JOIN
                        (SELECT name, gender, best_year from (select name, gender, yr as best_year, 
                        rank() OVER (PARTITION BY gender, name ORDER BY number DESC ) as ranked 
                        FROM names_popularity) ranking where ranked=1) b_year
                    ON aggregated.name=b_year.name
                    AND aggregated.gender=b_year.gender),
               
               cte1 as
                (SELECT cte.name, cte.gender, total_count, first_year, last_year, number_of_years, max_count, best_year,
                rank() over (partition by cte.gender order by total_count DESC) as ranked_total FROM cte)
       
               SELECT cte1.name, cte1.gender, total_count, first_year, last_year, number_of_years, max_count, 
               best_year, ranked_total FROM cte1
               WHERE ranked_total<=10"""
best_all_times=pd.read_sql(best_all_times, engine)
best_all_times

,name,gender,total_count,first_year,last_year,number_of_years,max_count,best_year,ranked_total
0,Jennifer,F,1248581.0,1970,2018,49,63603,1972,1
1,Jessica,F,1023910.0,1970,2018,49,55994,1987,2
2,Ashley,F,843230.0,1970,2018,49,54854,1987,3
3,Sarah,F,783293.0,1970,2018,49,28484,1982,4
4,Emily,F,748673.0,1970,2018,49,26539,1999,5
5,Amanda,F,741551.0,1970,2018,49,41786,1987,6
6,Elizabeth,F,733718.0,1970,2018,49,20744,1990,7
7,Melissa,F,607101.0,1970,2018,49,34052,1979,8
8,Stephanie,F,593299.0,1970,2018,49,24860,1990,9
9,Nicole,F,566704.0,1970,2018,49,22967,1985,10


### 6. The most popular unisex names

- to answer the question of which names can be considered as unisex names, I had to find names that occurred a similar number of times as male and female
- to achive this I set a variable 'spread' which is calculated by dividing female_count by male_count. The closer 'spread' to 1, the more similar the number of occurrences of a given name in the male and female versions
- I only wish to see names that occured over 1000 times

In [6]:
unisex_names="""WITH unisex_names as
                    (SELECT males_and_females.name, total_count_Female, total_count_Male, 
                    abs((1-total_count_Female/total_count_Male)) as spread
                        FROM 
                            (SELECT females.name, total_count_Female, total_count_Male FROM
                            (SELECT name, sum(number)as total_count_Female FROM names_popularity
                            WHERE gender='F'
                            GROUP BY name) as females
                        INNER JOIN
                            (SELECT name, sum(number)as total_count_Male FROM names_popularity
                            WHERE gender='M'
                            GROUP BY name) as males
                        ON females.name=males.name) males_and_females
                        ORDER BY spread ASC)
                SELECT unisex_names.name, total_count_Female, total_count_Male FROM unisex_names
                WHERE total_count_Female >1000"""
unisex_names=pd.read_sql(unisex_names, engine)
unisex_names.head(20)

,name,total_count_female,total_count_male
0,Emory,5287.0,5276.0
1,Yael,3361.0,3387.0
2,Ashten,1131.0,1150.0
3,Tristyn,2042.0,1988.0
4,Ocean,1707.0,1757.0
5,Shai,1231.0,1191.0
6,Landry,4036.0,4198.0
7,Marion,7960.0,7658.0
8,Quinn,31497.0,30302.0
9,Justice,16354.0,17110.0
